In [35]:
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta

In [2]:
# The subsidy folder (shared/homeworks/python_ds_miniprojects/5_subsid) contains files (tm_sales_1, tm_sales_2, ...)
# with product sales via telemarketing. Each file contains at least 4 columns (fields): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

# The essence of the task is to verify product connections to specific users by linking the sales files to the logs
# on connections in the system.

# Data peculiarities:

# Telemarketing employees do not always specify the full id, if the 'id' is not at the beginning of the SUBS_ID,
# you need to add it
# the fields in the files can be located completely randomly, but the field names are static
# sales are not counted if disconnection (END_DTTM) took place less than 5 minutes after connecting (START_DTTM)
# if a line without SUBS_ID is found in the sales file, it is skipped
# Save result to dataframe with ; separator, containing valid connections.

# Note: pay attention to the way pandas translates the date from the string, you might need the format parameter
# An example of content of the resulting file (column difference is the difference between START_DTTM and END_DTTM):

# ACT_DTTM;FILIAL_ID;difference
# 20-03-2020 14:59;1;255 days 09:01:00


In [3]:
sells_via_telemarketing = pd.DataFrame({})

for csv_path in Path('.').glob("**/tm_sales_*.csv"):
    small_df = pd.read_csv(str(csv_path), sep = ';')
    sells_via_telemarketing = pd.concat([sells_via_telemarketing, small_df], ignore_index=True)
    
sells_via_telemarketing = sells_via_telemarketing.drop_duplicates()

In [4]:
# if a line without SUBS_ID is found in the sales file (=NaN ib SUBS_ID), it is skipped
sells_via_telemarketing = sells_via_telemarketing.dropna()

In [5]:
# Telemarketing employees do not always specify the full id, if the 'id' is not at 
# the beginning of the SUBS_ID, you need to add it


def add_id(id):
    if id[:2] != 'id':
        id = 'id' + id
        return id
    else:
        return id

sells_via_telemarketing['SUBS_ID'] = sells_via_telemarketing.SUBS_ID.apply(add_id)
sells_via_telemarketing

SUBS_ID  FILIAL_ID  PROD_ID          ACT_DTTM
0  id4651830          1     1954  20-03-2020 14:59
1  id7646509          5     6431  19-03-2020 13:00
2  id7412683          4     3313  22-03-2020 17:25
3  id5416547          3     1743  17-03-2020 10:17
5  id8362218          7     9879  05-03-2020 11:42
6  id2185490          2     3210  16-03-2020 16:28
7  id5764122          3     1499  18-03-2020 15:44
8  id7642700          6     3020  15-03-2020 14:21
9  id1374509          2     5677  17-03-2020 11:48

In [54]:
path_to_logs = Path(Path.cwd(), '5_subsid', 'prod_activations_logs.csv')

In [55]:
prod_logs = pd.read_csv(path_to_logs, sep = ';', parse_dates = ['START_DTTM', 'END_DTTM'], dayfirst = True)
prod_logs

SUBS_ID PROD_ID          START_DTTM            END_DTTM
1  id4651830    1954 2020-03-20 14:59:00 2020-12-01 00:00:00
5  id7646509    6431 2020-03-19 13:00:00 2020-03-19 13:03:00
2  id7461794    3310 2020-03-20 17:25:00 2020-12-01 00:00:00
3  id5416547    1743 2020-03-17 10:17:00 2020-03-25 11:00:00
4  id8238421    1859 2020-03-01 11:42:00 2020-03-01 11:43:00
3  id8641743    2752 2020-03-20 15:44:00 2020-04-21 15:44:00
2  id2185490    3210 2020-03-16 16:28:00 2020-12-01 00:00:00
6  id7642700    3020 2020-03-15 14:21:00 2020-03-15 23:42:00
2  id8741631    5677 2020-03-19 12:28:00 2020-12-01 00:00:00

In [61]:
# sales are not counted if disconnection (END_DTTM) took place less than 5 minutes after connecting (START_DTTM)
prod_logs['time_delta_in_sec'] = (prod_logs['END_DTTM'] - prod_logs['START_DTTM']).dt.total_seconds()
prod_logs

SUBS_ID PROD_ID          START_DTTM            END_DTTM  \
1  id4651830    1954 2020-03-20 14:59:00 2020-12-01 00:00:00   
5  id7646509    6431 2020-03-19 13:00:00 2020-03-19 13:03:00   
2  id7461794    3310 2020-03-20 17:25:00 2020-12-01 00:00:00   
3  id5416547    1743 2020-03-17 10:17:00 2020-03-25 11:00:00   
4  id8238421    1859 2020-03-01 11:42:00 2020-03-01 11:43:00   
3  id8641743    2752 2020-03-20 15:44:00 2020-04-21 15:44:00   
2  id2185490    3210 2020-03-16 16:28:00 2020-12-01 00:00:00   
6  id7642700    3020 2020-03-15 14:21:00 2020-03-15 23:42:00   
2  id8741631    5677 2020-03-19 12:28:00 2020-12-01 00:00:00   

   time_delta_in_sec  
1         22064460.0  
5              180.0  
2         22055700.0  
3           693780.0  
4               60.0  
3          2764800.0  
2         22404720.0  
6            33660.0  
2         22159920.0

In [67]:
prod_logs['disconnect_avoided'] = prod_logs['time_delta_in_sec'] > (5 * 60.0)
prod_logs

SUBS_ID PROD_ID          START_DTTM            END_DTTM  \
1  id4651830    1954 2020-03-20 14:59:00 2020-12-01 00:00:00   
5  id7646509    6431 2020-03-19 13:00:00 2020-03-19 13:03:00   
2  id7461794    3310 2020-03-20 17:25:00 2020-12-01 00:00:00   
3  id5416547    1743 2020-03-17 10:17:00 2020-03-25 11:00:00   
4  id8238421    1859 2020-03-01 11:42:00 2020-03-01 11:43:00   
3  id8641743    2752 2020-03-20 15:44:00 2020-04-21 15:44:00   
2  id2185490    3210 2020-03-16 16:28:00 2020-12-01 00:00:00   
6  id7642700    3020 2020-03-15 14:21:00 2020-03-15 23:42:00   
2  id8741631    5677 2020-03-19 12:28:00 2020-12-01 00:00:00   

   time_delta_in_sec  disconnect_avoided  
1         22064460.0                True  
5              180.0               False  
2         22055700.0                True  
3           693780.0                True  
4               60.0               False  
3          2764800.0                True  
2         22404720.0                True  
6            33660.0                True  
2         22159920.0                True

In [84]:
prod_logs_disconnect_avoided = prod_logs.query("disconnect_avoided == True").SUBS_ID
sells_via_telemarketing_without_disconnect = sells_via_telemarketing.merge(prod_logs_disconnect_avoided, on = 'SUBS_ID')
sells_via_telemarketing_without_disconnect

SUBS_ID  FILIAL_ID  PROD_ID          ACT_DTTM
0  id4651830          1     1954  20-03-2020 14:59
1  id5416547          3     1743  17-03-2020 10:17
2  id2185490          2     3210  16-03-2020 16:28
3  id7642700          6     3020  15-03-2020 14:21

In [ ]:
# To check the results, enter the SUBS_ID from the resulting dataset in ascending order, separated by comma and space
# Answer: 2185490, 4651830, 5416547, 7642700
